In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
path_csi_hc = 'J:\\Data\\Wi-Fi_HC\\180_100\\'
path_save = "C:\\tmp\\"
# data info
df_info = pd.read_csv('data_subc_sig_v1.csv')
#df_info = df_info[df_info.id_person < 50]

person_uid = np.unique(df_info['id_person'])
csi_uid = np.unique(df_info['id'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))

# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_time = 15000 #int(np.max(df_info['len']))
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import h5py

In [3]:
file = np.unique(df_info.id.values)[0]
with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
    data1 = pickle.load(f)

In [4]:
data1.shape

(10258, 30, 2, 3)

In [5]:
data1_info = df_info[df_info.id==file].iloc[0]
np.array([data1_info.id_person,
          data1_info.id_location,
          data1_info.id_direction,
          data1_info.id_exp])

array([17,  1,  2, 11], dtype=int64)

In [6]:
# for dataset 1_1
csi_uid = np.unique(df_info[(df_info.id_location==1) & (df_info.id_direction==1)]['id'])
csi_uid

array(['S018_1_1_1', 'S018_1_1_10', 'S018_1_1_11', 'S018_1_1_12',
       'S018_1_1_2', 'S018_1_1_3', 'S018_1_1_5', 'S018_1_1_6',
       'S018_1_1_7', 'S018_1_1_8', 'S018_1_1_9', 'S019_1_1_1',
       'S019_1_1_11', 'S019_1_1_12', 'S019_1_1_2', 'S019_1_1_3',
       'S019_1_1_4', 'S019_1_1_5', 'S019_1_1_6', 'S019_1_1_8',
       'S019_1_1_9', 'S022_1_1_1', 'S022_1_1_10', 'S022_1_1_11',
       'S022_1_1_12', 'S022_1_1_2', 'S022_1_1_3', 'S022_1_1_4',
       'S022_1_1_6', 'S022_1_1_7', 'S022_1_1_8', 'S022_1_1_9',
       'S024_1_1_1', 'S024_1_1_10', 'S024_1_1_11', 'S024_1_1_12',
       'S024_1_1_2', 'S024_1_1_3', 'S024_1_1_4', 'S024_1_1_5',
       'S024_1_1_6', 'S024_1_1_7', 'S024_1_1_8', 'S024_1_1_9',
       'S025_1_1_1', 'S025_1_1_10', 'S025_1_1_11', 'S025_1_1_12',
       'S025_1_1_2', 'S025_1_1_3', 'S025_1_1_4', 'S025_1_1_5',
       'S025_1_1_6', 'S025_1_1_7', 'S025_1_1_8', 'S025_1_1_9',
       'S026_1_1_10', 'S026_1_1_11', 'S026_1_1_5', 'S026_1_1_6',
       'S026_1_1_7', 'S026_1_1_8', 'S02

In [7]:
len(csi_uid)

891

In [8]:
name_hdfs = 'csi_1_1.hdf5'
#Make HDF5
with h5py.File(path_save+name_hdfs,'w') as f:
    f.create_dataset('csi',(len(csi_uid),csi_time,csi_subc,2,3),dtype='complex128')
    f.create_dataset('label',(len(csi_uid),4),dtype='float32')
    csi_set = f['csi']
    label_set = f['label']
    
    for i,file in enumerate(csi_uid):
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
            # zero pad
            pad_len = csi_time - data1.shape[0]
            data1_pad = np.pad(data1,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

            # Label
            data1_info = df_info[df_info.id==file].iloc[0]
            data1_label = np.array([data1_info.id_person,
                      data1_info.id_location,
                      data1_info.id_direction,
                      data1_info.id_exp])
        label_set[i] = data1_label.astype('float32')
        csi_set[i] = data1_pad

In [11]:
f1 = h5py.File(path_save+'csi_1_1.hdf5','r')

In [12]:
f1['label'][0]

array([18.,  1.,  1.,  1.], dtype=float32)